<a href="https://colab.research.google.com/github/Quartz1605/PyTorch-Learnings/blob/main/Pytorch_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Hello pratham")

Hello pratham


In [3]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 23.3 MB/s eta 0:00:00


In [4]:
import optuna
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns =  ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

data_1 = pd.read_csv(url,names=columns)
data_1.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## Dealing with missing values (Preprocessing)

In [5]:
import numpy as np
columns_with_missing_values = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

data_1[columns_with_missing_values] =  data_1[columns_with_missing_values].replace(0,np.nan)



In [6]:
data_1.isnull().sum()

,0
Pregnancies,0
Glucose,5
BloodPressure,35
SkinThickness,227
Insulin,374
BMI,11
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [7]:
## Filling those values with mean of the other data

data_1.fillna(data_1.mean(),inplace=True) # data_1.mean() calculates mean table wise.
data_1.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [8]:
X = data_1.iloc[:,:8].values
y = data_1.iloc[:,8].values

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [10]:
scaler = StandardScaler();

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [11]:
X_train.shape

(537, 8)

## Optuna begins

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier


def objective(trail):

  estimators = trail.suggest_int('n-estimators',50,200) # suggest_int se samjhta hai which value to work on next from the sampler, sampler decides by looking at past value or past data.
  max_depth = trail.suggest_int('max_depth',5,20) # Defining range jisme check karna hai for max accuracy


  model = RandomForestClassifier(estimators,max_depth=max_depth,random_state=42)

  #Perfom a 3 fold cross-validation and calculate accuracy.
  score = cross_val_score(model,X_train,y_train,cv=3,scoring='accuracy').mean()

  return score #Return the accuracy score for Optuna to maximize

In [13]:
# create a study object(Experiment) and optimize the objective functiom

study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler())
study.optimize(objective,n_trials=50)



[I 2025-06-29 05:40:44,234] A new study created in memory with name: no-name-e327461a-5ac2-4e54-9c94-a8885ab28f76
[I 2025-06-29 05:40:44,893] Trial 0 finished with value: 0.7616387337057727 and parameters: {'n-estimators': 183, 'max_depth': 8}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-06-29 05:40:45,516] Trial 1 finished with value: 0.7616387337057727 and parameters: {'n-estimators': 170, 'max_depth': 9}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-06-29 05:40:46,062] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n-estimators': 150, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-06-29 05:40:46,347] Trial 3 finished with value: 0.7672253258845437 and parameters: {'n-estimators': 75, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-06-29 05:40:46,794] Trial 4 finished with value: 0.7616387337057727 and parameters: {'n-estimators': 122, 'max_depth': 13}. Best is trial 2 with value: 0.7728119

In [14]:
print(study.best_trial.value)
print(study.best_trial.params)

0.7802607076350093
{'n-estimators': 56, 'max_depth': 7}


## Making the best model accordingly

In [15]:
best_model = RandomForestClassifier(n_estimators=56,max_depth=7,random_state=42)

best_model.fit(X_train,y_train)

RandomForestClassifier(max_depth=7, n_estimators=56, random_state=42)

In [16]:
y_pred = best_model.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.7316017316017316

## Optuna visualizations

In [21]:
from optuna.visualization import plot_optimization_history,plot_parallel_coordinate,plot_contour,plot_param_importances,plot_slice

In [19]:
# 1. Optimization history, isse aap pata kar sakte ho how many trails are required.
plot_optimization_history(study).show()

In [20]:
plot_parallel_coordinate(study).show()

In [22]:
plot_slice(study).show()

In [23]:
plot_contour(study).show()

In [24]:
plot_param_importances(study).show()

## Selecting th best model or Model Hypertuning also known as Dynamic Search Spaces.

In [25]:
# Importing the necessary libraries.
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC

In [26]:
def objective(trial):

  # Choose the algo to tune.
  classifier_name = trial.suggest_categorical('classifier',['SVM','RandomForest','GradientBoosting'])

  if classifier_name == 'SVM':

    #SVM hyperparameter tuning

    c = trial.suggest_float('C',0.1,100,log=True)
    kernel = trial.suggest_categorical('kernel',['linear','rbf','poly','sigmoid'])
    gamma = trial.suggest_categorical('gamma',['scale','auto'])

    model = SVC(C=c,kernel=kernel,gamma=gamma,random_state=42)

  elif classifier_name == "RandomForest":

    n_estimators =  trial.suggest_int('n_estimators',50,300)
    max_depth = trial.suggest_int('max_depth',3,20)
    min_samples_split = trial.suggest_int('min_samples_split',2,10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,10)
    bootstrap = trial.suggest_categorical('bootstrap',[True,False])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

  elif classifier_name == "GradientBoosting":

    n_estimators =  trial.suggest_int('n_estimators',50,300)
    learning_rate = trial.suggest_float('learning_rate',0.01,0.3,log=True)
    max_depth = trial.suggest_int('max_depth',3,20)
    min_samples_split = trial.suggest_int('min_samples_split',2,10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,10)


    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

  score = cross_val_score(model,X_train,y_train,cv=3,scoring='accuracy').mean()

  return score




In [28]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)

[I 2025-06-29 06:13:50,022] A new study created in memory with name: no-name-aa8bc0a1-0c78-4ee8-9b1f-b76795fbd76d
[I 2025-06-29 06:13:50,833] Trial 0 finished with value: 0.7616387337057727 and parameters: {'classifier': 'RandomForest', 'n_estimators': 90, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-06-29 06:13:51,293] Trial 1 finished with value: 0.7597765363128491 and parameters: {'classifier': 'RandomForest', 'n_estimators': 133, 'max_depth': 17, 'min_samples_split': 3, 'min_samples_leaf': 7, 'bootstrap': False}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-06-29 06:13:51,320] Trial 2 finished with value: 0.7132216014897579 and parameters: {'classifier': 'SVM', 'C': 12.985336257727539, 'kernel': 'poly', 'gamma': 'scale'}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-06-29 06:13:51,934] Trial 3 finished with value: 0.7616387337057727 and parameters: {'classifier': 'Ra

In [29]:
#  0.7895716945996275 and parameters: {'classifier': 'SVM', 'C': 0.1306585209989422, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 15 with value: 0.7895716945996275.
best_trial = study.best_trial
best_trial.params

{'classifier': 'SVM',
 'C': 0.1306585209989422,
 'kernel': 'linear',
 'gamma': 'auto'}

In [30]:
best_trial.value

0.7895716945996275

In [32]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.761639,2025-06-29 06:13:50.023852,2025-06-29 06:13:50.833336,0 days 00:00:00.809484,NaN,True,RandomForest,NaN,NaN,NaN,20.0,4.0,8.0,90.0,COMPLETE
1,1,0.759777,2025-06-29 06:13:50.833964,2025-06-29 06:13:51.293493,0 days 00:00:00.459529,NaN,False,RandomForest,NaN,NaN,NaN,17.0,7.0,3.0,133.0,COMPLETE
2,2,0.713222,2025-06-29 06:13:51.294240,2025-06-29 06:13:51.320926,0 days 00:00:00.026686,12.985336,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.761639,2025-06-29 06:13:51.321640,2025-06-29 06:13:51.934459,0 days 00:00:00.612819,NaN,False,RandomForest,NaN,NaN,NaN,10.0,8.0,2.0,181.0,COMPLETE
4,4,0.739292,2025-06-29 06:13:51.935120,2025-06-29 06:13:51.953615,0 days 00:00:00.018495,4.227418,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.787709,2025-06-29 06:14:14.462511,2025-06-29 06:14:14.485962,0 days 00:00:00.023451,0.160325,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.787709,2025-06-29 06:14:14.486678,2025-06-29 06:14:14.510271,0 days 00:00:00.023593,0.100757,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.767225,2025-06-29 06:14:14.510735,2025-06-29 06:14:14.536470,0 days 00:00:00.025735,0.230071,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2025-06-29 06:14:14.536910,2025-06-29 06:14:14.554548,0 days 00:00:00.017638,0.191923,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [33]:
study.trials_dataframe()["params_classifier"].value_counts()

,count
params_classifier,
SVM,78
RandomForest,12
GradientBoosting,10
